In [1]:
import numpy as np
import pandas as pd
import talib
import numpy
import datetime as dt
# pd.set_option('display.max_rows', None)  # show all rows
# pd.set_option('display.max_columns', None)  # show all columns
# pd.set_option('display.width', None)  # don't wrap lines
# pd.set_option('display.max_colwidth', None)  # show full text in cells

df = pd.read_excel(r'gbp_usd_1h.xlsx')
df['atr'] = talib.ATR(
    df['high'],
    df['low'],
    df['close'],
    timeperiod=24
)
df['ma'] = talib.KAMA(
    df['close'],
    timeperiod=14
)

df['adx'] = talib.ADX(
    df['high'],
    df['low'],
    df['close'],
    timeperiod=24
)

df['rsi'] = talib.RSI(
    df['close'],
    timeperiod=14
)

df['macd'] = talib.RSI(
    df['close'],
    timeperiod=14
)


In [2]:
df = df.iloc[47:].reset_index(drop=True)

In [3]:
df = df.drop(columns=['Unnamed: 0'])

In [4]:
def set_ma_status(row):
    if row['close'] > row['ma']:
        return 'upper'
    if row['close'] < row['ma']:
        return 'under'
    return 'eq'


df['ma_status'] = df.apply(set_ma_status, axis=1)

In [5]:
df['id'] = df.index

In [6]:
import numpy as np

In [7]:
df['entry'] = df['close']
df['tp_buy'] = df['close'] + (df['atr'] * 2.4)
df['tp_sell'] = df['close'] - (df['atr'] * 2.4)

In [8]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['timestamp'] = df['datetime'].apply(lambda d: int(d.timestamp()))

In [9]:
array = np.array(df[['timestamp', 'open', 'high', 'low', 'close', 'atr', 'adx', 'tp_buy', 'tp_sell']])

In [10]:
patterns = []
array_size = len(array)

for i, element in enumerate(array):
    if i + 7 == array_size:
        break

    pat = array[i:i + 7, :]
    patterns.append(pat)

In [11]:
def get_trade_from_pattern(pattern: np.array):
    timestamp = pattern[-1, 0].copy()
    entry = pattern[-1, 4].copy()
    tp_buy = pattern[-1, 7].copy()
    tp_sell = pattern[-1, 8].copy()

    return timestamp, entry, tp_buy, tp_sell


any_trade = get_trade_from_pattern(patterns[10])

In [12]:
def evaluate_trade_result(chart: np.array, trade: tuple):
    timestamp, entry, tp_buy, tp_sell = trade

    backtest_chart = chart[chart[:, 0] >= timestamp]

    for i, element in enumerate(backtest_chart):
        price_high = element[2]
        price_low = element[3]

        # print(f'entry: {entry} price_high: {price_high} price_low: {price_low}')
        # print(f'tp_buy: {tp_buy} tp_sell: {tp_sell}')

        if price_high > tp_buy:
            return "buy"

        if price_low < tp_sell:
            return "sell"

    return "None"

In [13]:
trade_result = []
for pat in patterns:
    trade = get_trade_from_pattern(pat)
    result = evaluate_trade_result(array, trade)
    trade_result.append((trade[0], result))

In [14]:
patterns_for_train = []
for pat in patterns:
    id = pat[-1, 0]
    closes = pat[:, 4]
    atr = pat[-1, 5]
    adx = pat[-1, 6]
    result = np.array([id, atr, adx])
    result = np.hstack((result, closes))
    patterns_for_train.append(result)

In [15]:
patterns_for_train_df = pd.DataFrame(patterns_for_train, columns=[
    'timestamp',
    'atr',
    'adx',
    'close_1',
    'close_2',
    'close_3',
    'close_4',
    'close_5',
    'close_6',
    'close_7',
])

In [16]:
trade_result_df = pd.DataFrame(trade_result, columns=['timestamp', 'result'])

In [17]:
patterns_for_train_df = pd.merge(patterns_for_train_df, trade_result_df, on='timestamp')

In [18]:
df_ma_statuses = df[['timestamp', 'ma_status', 'macd', 'rsi']]

In [19]:
final_patterns_for_training = pd.merge(patterns_for_train_df, df_ma_statuses, on='timestamp')

In [20]:
final_patterns_for_training = final_patterns_for_training[
    final_patterns_for_training['result'] != 'None'
    ]

In [21]:
final_patterns_for_training['result'] = (
    final_patterns_for_training['result'].apply(lambda x: 1 if x == 'buy' else 0)
)

In [22]:
final_patterns_for_training['ma_status'] = (
    final_patterns_for_training['ma_status'].apply(lambda x: 1 if x == 'upper' else 0)
)

In [23]:
final_patterns_for_training.columns

Index(['timestamp', 'atr', 'adx', 'close_1', 'close_2', 'close_3', 'close_4',
       'close_5', 'close_6', 'close_7', 'result', 'ma_status', 'macd', 'rsi'],
      dtype='object')

In [24]:
data_set = final_patterns_for_training[
    ['atr', 'adx', 'ma_status', 'macd', 'rsi', 'close_1', 'close_2', 'close_3', 'close_4', 'close_5',
     'close_6', 'close_7', 'result']].copy()

In [25]:
data_set_array = np.array(data_set)
np.random.shuffle(data_set_array)

In [26]:
data_set_array.shape

(35521, 13)

In [27]:
X , y = data_set_array[:, :-1], data_set_array[:, -1]

In [28]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.2, train_size=0.8,random_state=42)

In [ ]:
from lightgbm import LGBMClassifier

lgbm_classifier = LGBMClassifier(

)